## Dependencies

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx # for graph creation
import time # for timing
from itertools import permutations # for generating all possible permutations


## Graph Creation

In [ ]:
class Graph():
    def __init__(self, vertices):
        # Initialize the graph as an adjacency matrix with all with 0's
        self.graph = [[0 for column in range(vertices)] for row in range(vertices)]
        self.V = vertices  # Set the number of vertices

    def add_edge(self, u, v):
        # Add an undirected edge between u and v
        self.graph[u][v] = 1
        self.graph[v][u] = 1

    def add_edges_from_graph(self, graph):
        for edge in graph.edges():
            self.add_edge(edge[0], edge[1])

    def summary(self):
        # print nodes and edges
        print("Nodes: ", self.V)
        print("Edges: ", sum([sum(row) for row in self.graph])//2)

    def plot_graph(self):
        G = nx.Graph()
        for i in range(self.V):
            G.add_node(i)
        for i in range(self.V):
            for j in range(i, self.V):
                if self.graph[i][j] == 1:
                    G.add_edge(i, j)
        nx.draw(G, with_labels=True)
        plt.show()

In [ ]:
# make graph for testing
# n = 10
# cycle_graph = nx.complete_graph(n)
# cycle_graph_instance = Graph(n)
# cycle_graph_instance.add_edges_from_graph(cycle_graph)
# cycle_graph_instance.summary()

## Brute Force

In [ ]:
def is_valid_cycle(perm, adj):
    # Check if the permutation forms a valid cycle in the graph
    N = len(perm)
    # Check if all consecutive vertices are connected
    for i in range(N - 1):
        if adj[perm[i]][perm[i + 1]] == 0:
            return False
    # Check if the last vertex is connected to the first vertex to complete the cycle
    if adj[perm[N-1]][perm[0]] == 0:
        return False
    return True

def hamiltonian_cycle_bruteforce(graph):
    # Find a Hamiltonian cycle in the graph using brute force
    adj = graph.graph if isinstance(graph, Graph) else graph  # Assuming Graph class has a 'graph' attribute
    N = len(adj)  # Number of vertices
    all_perms = permutations(range(N))
    
    for perm in all_perms:
        if is_valid_cycle(perm, adj):
            complete_cycle = perm + (perm[0],)
            # print_bf(complete_cycle)
            # plot_bf(graph, complete_cycle)
            return True  # Return True and the complete cycle
    # plot_bf(graph)
    # print("Solution does not exist\n")
    return False  # Return False if no valid cycle is found

# def print_bf(path):
#     print("Solution BF Exists: Following is one Hamiltonian Cycle")
#     for vertex in path[:-1]:
#         print(vertex, end=" -> ")
#     print(path[-1])

# def plot_bf(graph, path=None):
#     G = nx.Graph()
#     for i in range(len(graph)):
#         for j in range(i, len(graph)):
#             if graph[i][j] == 1:
#                 G.add_edge(i, j)
#     pos = nx.spring_layout(G)
#     plt.figure(figsize=(5, 2))
#     nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='yellow', edge_color='gray')
#     if path:
#         edges_in_path = [(path[i], path[(i + 1) % len(graph)]) for i in range(len(graph))]
#         nx.draw_networkx_edges(G, pos, edgelist=edges_in_path, edge_color='red', width=2)
#     plt.show()

In [ ]:
hamiltonian_cycle_bruteforce(cycle_graph_instance.graph)

## Back Tracking

In [ ]:
def isSafe(graph, v, pos, path):
    # Check if current vertex and last vertex in path are adjacent
    if graph[path[pos-1]][v] == 0:
        return False
    # Check if current vertex not already in path
    for vertex in path:
        if vertex == v:
            return False
    return True

def hamCycleUtil(graph, path, pos):
    V = len(graph)
    # Base case: if all vertices are included in the path
    if pos == V:
        # Last vertex must be adjacent to the first vertex in path to make a cycle
        if graph[path[pos-1]][path[0]] == 1:
            return True
        else:
            return False
    # Explore all vertices other than the first vertex
    for v in range(1, V):
        if isSafe(graph, v, pos, path):
            path[pos] = v
            if hamCycleUtil(graph, path, pos+1):
                return True
            # DeadEnd - Remove current vertex if it doesn't lead to a solution
            path[pos] = -1
    return False

def hamitonian_cycle_backtracking(graph):
    V = len(graph)
    # Initialize path as -1
    path = [-1] * V
    # Start from vertex 0 as the first vertex in the path
    path[0] = 0
    # Call the recursive helper function to find Hamiltonian cycle
    if not hamCycleUtil(graph, path, 1):
        # print("Solution does not exist\n")
        # plot_bt(graph)
        return False
    # print_bt(path)
    # plot_bt(graph, path)
    return True

# def print_bt(path):
#     print("Solution BT Exists: Following is one Hamiltonian Cycle")
#     for vertex in path:
#         print(vertex, end=' -> ')
#     print(path[0])  # Print the first vertex again to show the complete cycle

# def plot_bt(graph, path=None):

#     G = nx.Graph()
#     for i in range(len(graph)):
#         for j in range(i, len(graph)):
#             if graph[i][j] == 1:
#                 G.add_edge(i, j)
#     pos = nx.spring_layout(G)
#     plt.figure(figsize=(5, 2))
#     nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='yellow', edge_color='gray')
#     if path:
#         edges_in_path = [(path[i], path[(i + 1) % len(graph)]) for i in range(len(graph))]
#         nx.draw_networkx_edges(G, pos, edgelist=edges_in_path, edge_color='red', width=2)
#     plt.show()


In [ ]:
hamitonian_cycle_backtracking(cycle_graph_instance.graph)

## Dynamic Programming

In [ ]:
def Hamiltonian_cycle_dp(graph):
    """
    Check for a Hamiltonian cycle in a graph using dynamic programming.
    """
    adj = graph.graph if isinstance(graph, Graph) else graph  # Assuming Graph class has a 'graph' attribute
    N = len(adj)  # Number of vertices

    dp = [[False for _ in range(1 << N)] for _ in range(N)]
    parent = [[-1 for _ in range(1 << N)] for _ in range(N)]

    # Set dp[i][1 << i] to true for each node i
    for i in range(N):
        dp[i][1 << i] = True

    # Iterate over each subset of nodes
    for mask in range(1 << N):
        for j in range(N):
            if mask & (1 << j):
                for k in range(N):
                    if mask & (1 << k) and adj[k][j] == 1 and j != k and dp[k][mask ^ (1 << j)]:
                        dp[j][mask] = True
                        parent[j][mask] = k
                        break

    # Check for any node 'i' that can end a Hamiltonian cycle
    for i in range(N):
        if dp[i][(1 << N) - 1] and adj[i][0] == 1:
            # Recover the path
            path = []
            cur = i
            mask = (1 << N) - 1

            while cur != -1:
                path.append(cur)
                temp = cur
                cur = parent[cur][mask]
                mask = mask ^ (1 << temp)

            path.reverse()

            # Check if path forms a cycle
            if len(path) == N and adj[path[-1]][path[0]] == 1:
                path.append(path[0])
                # print_dp(path)
                # plot_dp(graph, path)
                return True
    # plot_dp(graph)
    # print("No Hamiltonian Cycle exists")
    return False

# def print_dp(path):
#     print("Solution DP Exists: Following is one Hamiltonian Cycle")
#     for vertex in path[:-1]:
#         print(vertex, end=' -> ')
#     print(path[-1])
    
# def plot_dp(graph, path=None):
#     G = nx.Graph()
#     for i in range(len(graph)):
#         for j in range(i, len(graph)):
#             if graph[i][j] == 1:
#                 G.add_edge(i, j)
#     pos = nx.spring_layout(G)
#     plt.figure(figsize=(5, 2))
#     nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='yellow', edge_color='gray')
#     if path:
#         edges_in_path = [(path[i], path[(i + 1) % len(graph)]) for i in range(len(graph))]
#         nx.draw_networkx_edges(G, pos, edgelist=edges_in_path, edge_color='red', width=2)
#     plt.show()


In [ ]:
Hamiltonian_cycle_dp(cycle_graph_instance.graph)

## Complexity Comparison

In [336]:
# Below code was reffered from chatgpt

# Function to measure execution time and detect cycle
def measure_runtime(func, graph):
    start_time = time.time()
    found = func(graph)  # Ensure your function returns True if cycle is found, False otherwise
    end_time = time.time()
    runtime = end_time - start_time
    return found, runtime

# Variables to store runtimes and graph types
graph_types = ['Complete', 'Random', 'Sparse']
algorithms = [hamiltonian_cycle_bruteforce, hamitonian_cycle_backtracking, Hamiltonian_cycle_dp]
runtime_data = {alg.__name__: {g_type: [] for g_type in graph_types} for alg in algorithms}
results_data = {alg.__name__: {g_type: [] for g_type in graph_types} for alg in algorithms}

# Loop to test different graph types and algorithms
for g_type in graph_types:
    for n in range(2, 13):  # Change range as needed
        if g_type == 'Complete':
            cycle_graph = nx.complete_graph(n)
        elif g_type == 'Random':
            cycle_graph = nx.erdos_renyi_graph(n, 0.5)  # Adjust the probability as needed
        elif g_type == 'Sparse':
            cycle_graph = nx.random_tree(n)

        cycle_graph_instance = Graph(n)
        cycle_graph_instance.add_edges_from_graph(cycle_graph)

        for alg in algorithms:
            found, runtime = measure_runtime(alg, cycle_graph_instance.graph)
            runtime_data[alg.__name__][g_type].append(runtime)
            results_data[alg.__name__][g_type].append(found)

# Plotting the runtimes for each algorithm and graph type
fig, axes = plt.subplots(len(algorithms), len(graph_types), figsize=(15, 10))

for i, alg in enumerate(algorithms):
    for j, g_type in enumerate(graph_types):
        x_values = range(2, 12)
        y_values = runtime_data[alg.__name__][g_type]
        results = results_data[alg.__name__][g_type]
        colors = ['red' if not result else 'blue' for result in results]
        
        axes[i, j].scatter(x_values, y_values, color=colors, label=f'{g_type} - Runtime', marker='x')
        axes[i, j].plot(x_values, y_values, linestyle='-', color='gray')
        axes[i, j].set_xlabel('Number of Vertices')
        axes[i, j].set_ylabel('Time Taken (seconds)')
        axes[i, j].set_title(f'{alg.__name__} - {g_type} Graphs')
        axes[i, j].legend()
        axes[i, j].grid(True)
# increase space between plots


plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# Create DataFrame from runtime_data dictionary
df_data = []
for alg_name, alg_data in runtime_data.items():
    for g_type, runtime_list in alg_data.items():
        for n, runtime in enumerate(runtime_list, start=2):
            df_data.append({
                'Algorithm': alg_name,
                'Graph Type': g_type,
                'Vertices': n,
                'Time Taken (seconds)': runtime
            })

# Create DataFrame
df = pd.DataFrame(df_data)

# Pivot DataFrame to have a proper table format
df_pivot = df.pivot_table(index=['Algorithm', 'Vertices'], columns='Graph Type', values='Time Taken (seconds)')

# Sort columns based on the specified order
df_pivot = df_pivot.reindex(columns=['Complete', 'Random', 'Sparse'], level=1)


In [ ]:
df_pivot